## Pre-processing for LMFM or FMTV ITU table queries

In [2]:
import geopandas as gpd
import pandas as pd
import glob

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


#### 0. Select ITU table and read extracts

In [3]:
# Choose table
table = input("Type: 'lfmf' or 'fmtv':")

if table == 'lfmf':
    file = '14_countries_lfmf.html'
elif table == 'fmtv':
    file_list = glob.glob(f"0. itu_queries/input/{table}/*.html")
    for i, value in enumerate(file_list):
        print(i, value)
    file_number = int(input('Select file: '))
    file = file_list[file_number]
else:
    print(f'{table} does not exist.')

0 0. itu_queries/input/fmtv/fmtv_benin_togo_burundi.html


Read File

In [64]:
df_table = pd.read_html(file, skiprows=0, header=0)[0]
df_table.ctry.unique()

array(['BDI', 'BEN', 'TGO'], dtype=object)

In [5]:
# Create a dictionary to map the ITU country codes to the full country names
country_dict = {
       'AGL': 'angola',
       'BDI':'burundi',
       'BEN':'benin',
       'COD':'drc',
       'ETH':'ethiopia',
       'GMB':'gambia',
       'IND':'india',
       'MLI':'mali',
       'MTN':'mauritania',
       'NGR':'niger',
       'NIG':'nigeria',
       'RRW':'rwanda',
       'SEN':'senegal',
       'TGO':'togo'
       }

### I. Clean and format ITU data for CloudRF Queries


* fill missing call signs with unique identifier
* rename columns
* update power to antennae from kW to W

_fmtv_:
* remove tv records by selecting those where `stn_cls` (station class) == `BC`
    * BC Broadcasting station, sound
    * BT Broadcasting station, television

_sanity checks_:
* remove negative power and frequencies below 1

In [65]:
# Drop televisions
df = df_table.loc[df_table.stn_cls == 'BC'].copy()

print(f"{len(df)} of {len(df_table)} are FM stations.")

825 of 1313 are FM stations.


In [66]:
# Check columns
sorted(df.columns.sort_values())

['addr_code',
 'adm',
 'adm_ref_id',
 'amend_type',
 'ant_dir',
 'are_rmk_conds_met',
 'assgn_code',
 'assgn_id',
 'assoc_allot_id',
 'assoc_allot_sfn_id',
 'azm_max_e',
 'bdwdth',
 'beam_tilt_angle',
 'call_sign',
 'color',
 'ctry',
 'd_adm_ntc',
 'd_expiry',
 'd_inuse',
 'd_pub_req',
 'd_rcv',
 'd_recog',
 'd_updated',
 'eff_hgtmax',
 'emi_cls',
 'erp_beam_tilt_dbw',
 'erp_dbw',
 'erp_h_dbw',
 'erp_h_kw',
 'erp_kw',
 'erp_v_dbw',
 'erp_v_kw',
 'eu',
 'eu_ref',
 'fdg_observ',
 'fragment',
 'freq_assgn',
 'freq_block',
 'freq_max',
 'freq_min',
 'freq_scarr',
 'freq_stabl',
 'freq_vcarr',
 'has_signed_commitment',
 'hgt_agl',
 'intent',
 'is_digital',
 'is_notice',
 'is_pub_req',
 'is_public',
 'is_resub',
 'lat_dec',
 'lat_deg',
 'lat_min',
 'lat_ns',
 'lat_rdn',
 'lat_sec',
 'long_dec',
 'long_deg',
 'long_ew',
 'long_min',
 'long_rdn',
 'long_sec',
 'mod_type',
 'notice_typ',
 'offset_khz',
 'op_agcy',
 'oset_s_12',
 'oset_s_khz',
 'oset_v_12',
 'oset_v_khz',
 'plan_adm_ref_id',
 'p

In [67]:
# fill null call_sign with station id and terrakey so that it can be used for identification
df.call_sign = df.call_sign.fillna(df.stn_id).fillna(df.terrakey)

# rename columns
if table == 'lfmf':
    df.rename(columns = {
        'site_name': 'site',
        'call_sign': 'network',
        'pwr_kw' : 'transmitter.txw',
        'freq_assgn': 'transmitter.frq', #assigned frequency in kHz
        'ant_hgt_m' : 'transmitter.alt',
        'long_dec': 'transmitter.lon',
        'lat_dec': 'transmitter.lat',
        'azm_max_e': 'antenna.azi', # azimuth
    },
        inplace = True)

elif table == 'fmtv':
    df.rename(columns = {
        'site_name': 'site',
        'call_sign': 'network',
        'erp_kw' : 'transmitter.txw',
        'freq_assgn': 'transmitter.frq', 
        'hgt_agl' : 'transmitter.alt',
        'long_dec': 'transmitter.lon',
        'lat_dec': 'transmitter.lat',
        'azm_max_e': 'antenna.azi', # azimuth
        # 'beam_tilt_angle': 'antenna.tilt', #Antenna tilt is mostly null
        'polar': 'antenna.pol'

    },
        inplace = True)

    # convert polarization to lower case
    df['antenna.pol'] = df['antenna.pol'].str.lower()

    # add Antenna pattern code:
    # 0 - Allows custom options (see: hbw; vbw; fbr)
    # 1 - Vertical dipole (omni-directional)
    df['antenna.ant'] = df.ant_dir.map(lambda x: 0 if x == 'D' else 1)


# update power to antennae from kW to watts
df['transmitter.txw'] = df['transmitter.txw'] * 1000

# Data quality checks -----
# power
negatives = df.loc[df['transmitter.txw'] <= 0]
print(f'dropping {len(negatives)} records with transmitter power at or below 0')
df = df.loc[df['transmitter.txw'] > 0]
print(f'{len(df)} remaining records.')

# frequency
negatives = df.loc[df['transmitter.frq'] <= 1]
print(f'dropping {len(negatives)} records with transmitter frequency is at or below 1')
df = df.loc[df['transmitter.frq'] > 1]
print(f'{len(df)} remaining records.')


dropping 0 records with transmitter power at or below 0
825 remaining records.
dropping 0 records with transmitter frequency is at or below 1
825 remaining records.


### II. Imputations

* height
* azimuth

_lfmf_:
* beamwidth

In [68]:
# HEIGHTS ----
# Define imputation values for heights
print('Summary of height imputation values: ')
total = df['transmitter.alt'].mode()[0]
ll = df['transmitter.alt'].min()
ul = df['transmitter.alt'].max()
print(f'Overall mode: {total}\nLower limit: {ll}\nUpper limit: {ul}')


print(f"Imputing {len(df['transmitter.alt'].isnull())} heights...")
# impute mode
df.fillna({'transmitter.alt': total}, inplace=True)

# AZIMUTH ----
print(f"Imputing {len(df['antenna.azi'].isnull())} azimuth...")
# impute default antenna azi and hbw from fem.json (cloudrf template)
df.fillna({'antenna.azi': 0}, inplace =True)

if table == 'lfmf':
    print(f"Imputing {len(df['antenna.hbw'].isnull())} beamswidth...")
    df['antenna.hbw'] = 120

Summary of height imputation values: 
Overall mode: 30.0
Lower limit: 0.0
Upper limit: 210.0
Imputing 825 heights...
Imputing 825 azimuth...


Define columns to be included in the export

In [69]:
if table == 'lfmf':
    cols = [
        'site',
        'network',
        'transmitter.lat',
        'transmitter.lon',
        'transmitter.alt',
        'transmitter.txw',
        'transmitter.frq',
        'antenna.azi'
        ]

elif table == 'fmtv':
    cols = [
        'site',
        'network',
        'transmitter.lat',
        'transmitter.lon',
        'transmitter.alt',
        'transmitter.txw',
        'transmitter.frq',
        'antenna.azi',
        'antenna.pol'
        ]

Separate and export

In [70]:
for cty in df.ctry.unique():
    country = country_dict[cty]
    print(country)

    # separate dataframe by countr
    df_subset = df.loc[df.ctry == cty]

    # export full df for each country to future review
    df_subset.to_csv(f'0. itu_queries/output/{table}/{country}_{table}_full.csv', index=False)

    # export subset for cloudrf
    df_subset[cols].to_csv(f"../cloudrf/input/{table}/{country}_{table}.csv", index=False)

burundi
benin
togo


In [71]:
df.groupby('ctry').terrakey.count().reset_index()

,ctry,terrakey
0,BDI,200
1,BEN,473
2,TGO,152
